Chapter7　アプリケーションへの組込み（7.4節以降）

In [0]:
import torch
torch.tensor([1,2,3]).to("cuda:0")

In [0]:
import torch
from torch import nn, optim
from torch.utils.data import (Dataset,
                            DataLoader,
                            TensorDataset)
import tqdm

リスト7.12　学習済みモデルの読み込み

In [0]:
from torchvision import models

def create_network():
    
    # resnet18ベースの2クラス識別モデル
    net = models.resnet18()
    fc_input_dim = net.fc.in_features
    net.fc = nn.Linear(fc_input_dim, 2)
    return net

# モデルの生成
net = create_network()

# パラメータの読み込みとモデルへセット
prm = torch.load("taco_burrito.prm", map_location="cpu")
net.load_state_dict(prm)

# 評価モードに設定
net.eval()

リスト7.13　taco_burrito.onnxの出力

In [0]:
import torch.onnx
dummy_data = torch.empty(1, 3, 224, 224, dtype=torch.float32)
torch.onnx.export(net, dummy_data, "taco_burrito.onnx")

リスト7.14　ONNXからインポート

In [0]:
import onnx
from caffe2.python.onnx import backend as caffe2_backend

# ONNXモデルを読み込む
onnx_model = onnx.load("taco_burrito.onnx")

# ONNXモデルをCaffe2モデルに変換
backend = caffe2_backend.prepare(onnx_model)

リスト7.15　PyTorchモデルとONNX経由のCaffe2モデルの比較（<your_path>は任意のディレクトリを指定）

In [0]:
from PIL import Image
from torchvision import transforms

# 画像をクロップしてTensorに変換する関数
transform = transforms.Compose([
    transforms.CenterCrop(224),
    transforms.ToTensor()
])

# 画像の読み込み
img = Image.open("<your_path>/test/burrito/360.jpg")

# Tensorに変換し、バッチの次元を足す
img_tensor = transform(img).unsqueeze(0)

# ndarrayに変換
img_ndarray = img_tensor.numpy()

# PyTorchで実行
net(img_tensor)

In [0]:
# ONNX/Caffe2で実行
output = backend.run(img_ndarray)
output[0]

リスト7.16　ONNXに依存せずCaffe2のモデルに変換する

In [0]:
from caffe2.python.onnx.backend import Caffe2Backend

init_net, predict_net = \
    Caffe2Backend.onnx_graph_to_caffe2_net(onnx_model)

リスト7.17　生成したCaffe2のネットワーク定義とパラメータの保存

In [0]:
with open('init_net.pb', "wb") as fopen:
    fopen.write(init_net.SerializeToString())

with open('predict_net.pb', "wb") as fopen:
    fopen.write(predict_net.SerializeToString())